In [ ]:
# Export Vault Variables 

export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

In [ ]:
## Create a policy file named read.hcl for Bob the reader. ( accessor ) 

tee read.hcl <<EOF
path "aws/creds/my-role" {
  capabilities = [ "read" ]

  control_group = {
    factor "authorizer" {
      identity {
        group_names = [ "acct_manager" ]
        approvals = 1
      }
    }
  }
}
EOF

In [ ]:
## Create another policy file named, acct_manager.hcl. 
## This is the policy needed for the member of controller (acct_manager) to approve Bob's request

## The important thing here is that the authorizer must have create and update permission on 
## the sys/control-group/authorize endpoint so that they can approve the request

tee acct_manager.hcl <<EOF
# To approve the request
path "sys/control-group/authorize" {
    capabilities = ["create", "update"]
}

# To check control group request status
path "sys/control-group/request" {
    capabilities = ["create", "update"]
}
EOF


In [ ]:
## Create a new policy named read-cred 

vault policy write read-cred read.hcl

In [ ]:
## Create a new policy named acct_manager.

vault policy write acct_manager acct_manager.hcl
